In [36]:
import ast
import pandas as pd
import re

rows = []
with open('steam_games.json') as f: 
          for line in f.readlines():
            rows.append(ast.literal_eval(line))

df = pd.DataFrame(rows)

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


In [38]:
df.isnull().sum()

publisher          8052
genres             3283
app_name              2
title              2050
url                   0
release_date       2067
tags                163
discount_price    31910
reviews_url           2
specs               670
price              1377
early_access          0
id                    2
developer          3299
sentiment          7182
metascore         29458
dtype: int64

In [39]:
df1 = df.copy()

In [73]:
df1.isnull().sum()

genres            0
app_name          0
title             0
price             0
developer       155
metascore         0
release_year      0
dtype: int64

In [41]:
#relleno los nombres de title con app_name, ya que corroboré que no sean los mismos anteriormente
df1['title'].fillna(df1['app_name'], inplace=True)

In [53]:
verificar_nulos = ['app_name', 'title']
df1 = df1.dropna(subset=verificar_nulos)

In [42]:
#cambio el type de release_data
df1['release_date'] = pd.to_datetime(df1['release_date'], errors='coerce')

In [43]:
#intento rescatar los valores que no estan en date, esta función la voy a usar posteriormente para crear una columna solo con los años
def extract_year(text):
    match = re.search(r'\b\d{4}\b', str(text))
    if match:
        return int(match.group())
    return None

In [44]:
df1['release_year'] = df1['release_date'].apply(extract_year)

In [45]:
#los datos irrelevantes los borro, ya que considero que el año es la base de las consultas y lo que me va a permitir más adelante observar la variación en el tiempo (si la hay) de los precios por genero o developer
df1.dropna(subset=['release_year'], inplace=True)

In [46]:
df1.dropna(subset=['genres'], inplace=True)

In [47]:
columns_to_drop = ['url', 'tags', 'discount_price', 'reviews_url', 'release_date', 'publisher', 'specs', 'sentiment', 'early_access', 'id', 'app_name']
df1 = df1.drop(columns=columns_to_drop)

In [48]:
#antes de hacer esto, tengo 27168 nulos... por lo tanto, dudo mucho que metascore me sirva para lo que quiero hacer en un futuro, pero igualmente quise hacer algunos experimentos sin borrar los metascore.
df1['metascore'].fillna(0, inplace=True)

In [81]:
#en esta celda voy corroborando cuales son los datos vecinos de las filas en las que tengo nulos, para saber que otros datos estoy borrando al borrar esa fila.
filas_nulas_info= df1[df1['developer'].isnull()]
filas_nulas_info

,genres,title,price,developer,metascore,release_year


In [57]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28548 entries, 0 to 32133
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28548 non-null  object 
 1   app_name      28548 non-null  object 
 2   title         28548 non-null  object 
 3   price         27579 non-null  object 
 4   developer     28379 non-null  object 
 5   metascore     28548 non-null  object 
 6   release_year  28548 non-null  float64
dtypes: float64(1), object(6)
memory usage: 1.7+ MB


In [64]:
#hay varios valores como "Free Demo", "Free to Play", "Free To Play", lo cual me conviene que estén igualados a 0.
df1['price'] = df1['price'].replace('[^0-9]', '0', regex=True)

In [70]:
df1.dropna(subset=['price'], inplace=True)

In [74]:
#No quiero descartar los developers nulos, porque quizás me sirvan.
df1['developer'].fillna("Other", inplace=True)

In [82]:
df1.isnull().sum()

genres          0
title           0
price           0
developer       0
metascore       0
release_year    0
dtype: int64

In [79]:
#voy a guardar todos los cambios que hice en un json nuevo y de ahi voy a "experimentar" en otro ipynb.
eda_df = 'eda_df.json'
df1.to_json(eda_df, orient='records', lines=True)